## Signals
In Django, signals are a way to allow decoupled applications to get notified when certain actions occur elsewhere in the framework. They are especially useful for triggering actions in response to events such as model saves, deletions, or user logins.
The signals are implemented using the `django.dispatch` module, which provides a way to create and manage signals.

The signal methods name usually follow the pattern of `<model>_<action>`, where `<model>` is the name of the model and `<action>` is the event that triggers the signal (e.g., `pre_save`, `post_save`, etc.).

The most commonly used signals in Django include:
1. `pre_save` and `post_save`: Sent before or after a model's `save()` method is called.
    ```python
    from django.db.models.signals import pre_save, post_save
    from django.dispatch import receiver
    from myapp.models import MyModel
    @receiver(pre_save, sender=MyModel)
    def mymodel_pre_save(sender, instance, **kwargs):
        # Code to execute before saving MyModel instance
        pass
    @receiver(post_save, sender=MyModel)
    def mymodel_post_save(sender, instance, created, **kwargs):
        # Code to execute after saving MyModel instance
        if created:
            # Code for newly created instance
            pass
    ```
2. `pre_delete` and `post_delete`: Sent before or after a model's `delete()` method is called.
    ```python
    from django.db.models.signals import pre_delete, post_delete
    from django.dispatch import receiver
    from myapp.models import MyModel
    @receiver(pre_delete, sender=MyModel)
    def mymodel_pre_delete(sender, instance, **kwargs):
        # Code to execute before deleting MyModel instance
        pass
    @receiver(post_delete, sender=MyModel)
    def mymodel_post_delete(sender, instance, **kwargs):
        # Code to execute after deleting MyModel instance
        pass
    ```
3. `m2m_changed`: Sent when a ManyToManyField is changed.
    ```python
    from django.db.models.signals import m2m_changed
    from django.dispatch import receiver
    from myapp.models import MyModel
    @receiver(m2m_changed, sender=MyModel.my_many_to_many_field.through)
    def mymodel_m2m_changed(sender, instance, action, **kwargs):
        # Code to execute when ManyToManyField changes
        pass
    ```
4. `request_started` and `request_finished`: Sent when a request starts and finishes.This signal is applied to the entire Django application, not just a specific model or app and is useful for logging or monitoring purposes.
    ```python
    from django.core.signals import request_started, request_finished
    from django.dispatch import receiver
    @receiver(request_started)
    def request_started_handler(sender, **kwargs):
        # Code to execute when a request starts
        pass
    @receiver(request_finished)
    def request_finished_handler(sender, **kwargs):
        # Code to execute when a request finishes
        pass
    ```

### Authentication Signals
Django also provides signals related to user authentication, such as:
1. `user_logged_in`: Sent when a user logs in.
2. `user_logged_out`: Sent when a user logs out.
3. `user_login_failed`: Sent when a login attempt fails.
    ```python
    from django.contrib.auth.signals import user_logged_in, user_logged_out, user_login_failed
    from django.dispatch import receiver

    @receiver(user_logged_in)
    def user_logged_in_handler(sender, request, user, **kwargs):
        # Code to execute when a user logs in
        pass

    @receiver(user_logged_out)
    def user_logged_out_handler(sender, request, user, **kwargs):
        # Code to execute when a user logs out
        pass
    
    @receiver(user_login_failed)
    def user_login_failed_handler(sender, credentials, request, **kwargs):
        # Code to execute when a login attempt fails
        pass
    ```

## Importing Signals
To ensure that your signal handlers are connected when Django starts, you typically import the module containing your signal handlers in the `ready` method of your app's `AppConfig` class.
```python
from django.apps import AppConfig
class MyAppConfig(AppConfig):
    name = 'myapp'
    def ready(self):
        import myapp.signals  # Import the signals module to connect signal handlers
```

## Custom Signals
In addition to the built-in signals, you can also create your own custom signals using the `Signal` class from `django.dispatch`.
```python
from django.dispatch import Signal

# Define a custom signal
my_custom_signal = Signal(providing_args=["arg1", "arg2"])

# Connect a receiver to the custom signal
from django.dispatch import receiver
@receiver(my_custom_signal)
def my_custom_signal_receiver(sender, **kwargs):
    arg1 = kwargs.get('arg1')
    arg2 = kwargs.get('arg2')
    # Code to execute when the custom signal is sent
    pass

# Send the custom signal
my_custom_signal.send(sender=None, arg1='value1', arg2='value2')
```

## Advanced Signal Patterns
- Signal chaining
- Conditional signal handling
- Signal priorities
- Weak references to avoid memory leaks
### Example: Signal Chaining

In [ ]:
from django.dispatch import Signal
from django.dispatch import receiver

# Define signals
first_signal = Signal()
second_signal = Signal()

@receiver(first_signal)
def first_handler(sender, **kwargs):
    print('First signal received')
    second_signal.send(sender=sender)

@receiver(second_signal)
def second_handler(sender, **kwargs):
    print('Second signal received')

## Monitoring and Debugging Signals
- Log signal emissions
- Track signal performance
- Debug signal connections
### Example: Logging Signals

In [ ]:
import logging
from django.db.models.signals import post_save
from django.dispatch import receiver

logger = logging.getLogger(__name__)

@receiver(post_save)
def log_model_save(sender, instance, **kwargs):
    logger.info(f'Model {sender.__name__} saved: {instance}')

## Testing Signals
- Test signal emission
- Mock signal receivers
- Test signal order
### Example: Testing Signals

In [ ]:
from django.test import TestCase
from django.db.models.signals import post_save
from django.dispatch import receiver
from myapp.models import MyModel

class SignalTest(TestCase):
    def test_post_save_signal(self):
        # Disconnect receiver to avoid side effects
        post_save.disconnect(receiver=my_receiver, sender=MyModel)
        
        instance = MyModel.objects.create(name='test')
        # Test that signal was sent
        # You can use signal mocking or check side effects
        
        # Reconnect receiver
        post_save.connect(receiver=my_receiver, sender=MyModel)

## Best Practices
- Keep signal handlers lightweight
- Avoid circular signal dependencies
- Use weak references for receivers
- Document signal usage
- Test signal behavior
- Consider performance impact

## Real-World Examples
- Audit logging
- Cache invalidation
- Notification systems
- Background task triggering
### Example: Cache Invalidation

In [ ]:
from django.core.cache import cache
from django.db.models.signals import post_save, post_delete
from django.dispatch import receiver
from myapp.models import Article

@receiver([post_save, post_delete], sender=Article)
def invalidate_article_cache(sender, instance, **kwargs):
    cache.delete(f'article_{instance.id}')
    cache.delete('article_list')